In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc

In [2]:
from google.colab import drive
drive.mount('/content/drive') #ระวังเครื่องหมาย ‘ กับ ' ต่างกันนะคะ

Mounted at /content/drive


In [3]:
import pandas as pd
path = '/content/drive/MyDrive/Colab Notebooks/dataset/people-2000000.csv'
dfp = pd.read_csv(path) ## Dataset is now stored in a Pandas Dataframe
dfp

,Index,User Id,First Name,Last Name,Sex,Email,Phone,Date of birth,Job Title
0,1,4defE49671cF860,Sydney,Shannon,Male,tvang@example.net,574-440-1423x9799,2020-07-09,Technical brewer
1,2,F89B87bCf8f210b,Regina,Lin,Male,helen14@example.net,001-273-664-2268x90121,1909-06-20,"Teacher, adult education"
2,3,Cad6052BDd5DEaf,Pamela,Blake,Female,brent05@example.org,927-880-5785x85266,1964-08-19,Armed forces operational officer
3,4,e83E46f80f629CD,Dave,Hoffman,Female,munozcraig@example.org,001-147-429-8340x608,2009-02-19,Ship broker
4,5,60AAc4DcaBcE3b6,Ian,Campos,Female,brownevelyn@example.net,166-126-4390,1997-10-02,Media planner
...,...,...,...,...,...,...,...,...,...
1999995,1999996,AbBE4F3Eb5df843,Susan,Ellison,Male,yward@example.net,855.222.4135x62040,1913-01-05,Research scientist (maths)
1999996,1999997,e71af3E8De63CDF,Cole,Sherman,Female,nbarnett@example.org,(130)829-9159x800,1977-04-15,Marketing executive
1999997,1999998,fAF0618097acD1B,Ethan,Leonard,Male,brett57@example.com,514.898.2376x3069,1960-08-14,Photographer
1999998,1999999,f1FC3346F68DFAb,Jordan,Waller,Female,harveykendra@example.com,6920274656,1911-09-05,Chartered accountant


In [43]:
dfp.to_parquet('person.parquet')

In [41]:
table = pa.Table.from_pandas(dfp)
print(table)

pyarrow.Table
Index: int64
User Id: string
First Name: string
Last Name: string
Sex: string
Email: string
Phone: string
Date of birth: string
Job Title: string
----
Index: [[1,2,3,4,5,...,1999996,1999997,1999998,1999999,2000000]]
User Id: [["4defE49671cF860","F89B87bCf8f210b","Cad6052BDd5DEaf","e83E46f80f629CD","60AAc4DcaBcE3b6",...,"AbBE4F3Eb5df843","e71af3E8De63CDF","fAF0618097acD1B","f1FC3346F68DFAb","B04Bb8bE7DA70ba"]]
First Name: [["Sydney","Regina","Pamela","Dave","Ian",...,"Susan","Cole","Ethan","Jordan","Shelley"]]
Last Name: [["Shannon","Lin","Blake","Hoffman","Campos",...,"Ellison","Sherman","Leonard","Waller","Meza"]]
Sex: [["Male","Male","Female","Female","Female",...,"Male","Female","Male","Female","Female"]]
Email: [["tvang@example.net","helen14@example.net","brent05@example.org","munozcraig@example.org","brownevelyn@example.net",...,"yward@example.net","nbarnett@example.org","brett57@example.com","harveykendra@example.com","gamblejohnny@example.net"]]
Phone: [["574-440-1

In [48]:
# write to multiple partitioned files
pq.write_to_dataset(table, root_path='parq_file', partition_cols=['Job Title'])

In [49]:
table3 = pq.read_table('parq_file',columns=['Index', 'First Name','Last Name','Sex', 'Job Title'])
table3.to_pandas()

,Index,First Name,Last Name,Sex,Job Title
0,131521,Kaitlyn,Baxter,Female,Academic librarian
1,133596,Shelby,Henderson,Male,Academic librarian
2,133800,Angel,Wu,Female,Academic librarian
3,133935,Anthony,Haney,Female,Academic librarian
4,134472,Henry,Hatfield,Female,Academic librarian
...,...,...,...,...,...
1999995,1964579,Jaclyn,Lucas,Female,Youth worker
1999996,1965049,Philip,Cooley,Female,Youth worker
1999997,1965107,Kirsten,Frazier,Female,Youth worker
1999998,1965658,April,Griffin,Male,Youth worker


In [50]:
import os
import shutil


# Specify the partition directory
partition_directory = '/content/parq_file'

# List files in the partition directory
files = os.listdir(partition_directory)
print(files)


['Job Title=Investment analyst', 'Job Title=Investment banker, operational', 'Job Title=Waste management officer', 'Job Title=Medical secretary', 'Job Title=Cartographer', 'Job Title=Physicist, medical', 'Job Title=Naval architect', 'Job Title=Health and safety adviser', 'Job Title=Press photographer', 'Job Title=Research scientist (physical sciences)', 'Job Title=Fisheries officer', 'Job Title=Clothing', 'Job Title=Dramatherapist', 'Job Title=Garment', 'Job Title=Automotive engineer', 'Job Title=Administrator, sports', 'Job Title=Soil scientist', 'Job Title=Government social research officer', 'Job Title=Chemist, analytical', 'Job Title=Engineer, maintenance', 'Job Title=Producer, radio', 'Job Title=Leisure centre manager', 'Job Title=Counsellor', 'Job Title=Early years teacher', 'Job Title=Immunologist', 'Job Title=Editor, magazine features', 'Job Title=Community development worker', 'Job Title=Games developer', 'Job Title=Sound technician, broadcasting', 'Job Title=Farm manager', 'J

In [47]:
# Remove files and subdirectories in the partition directory
for item in files:
    item_path = os.path.join(partition_directory, item)
    if os.path.isfile(item_path):
        os.remove(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)  # Recursively remove subdirectories and their contents

# Remove the partition directory
os.rmdir(partition_directory)

In [60]:
import pyarrow.compute as pc
groupined_table = table3.group_by("Job Title").aggregate(
  [("Index",
  "count",  #count #max #min #sum

  pc.CountOptions(mode="only_valid"))]
)

groupined_table.to_pandas()

,Index_count,Job Title
0,3234,Academic librarian
1,3134,Accommodation manager
2,3185,"Accountant, chartered certified"
3,3130,"Accountant, chartered management"
4,3174,"Accountant, chartered public finance"
...,...,...
631,3092,Water quality scientist
632,3116,Web designer
633,2943,Wellsite geologist
634,3140,Writer


กรองข้อมูลโดยแสดงเฉพาะข้อมูลของคนที่เป็นเพศชายและมีตำแหน่งเป็น "Writer" เท่านั้น

In [56]:
expr1 = pc.field("Sex") == "Male"
expr2 = pc.field("Job Title") == "Writer"
# table3.filter(expr)
table3.select(['Last Name', 'Sex','Job Title']).filter(expr1 & expr2).to_pandas()

,Last Name,Sex,Job Title
0,Cherry,Male,Writer
1,Melton,Male,Writer
2,Zhang,Male,Writer
3,Harrison,Male,Writer
4,Sellers,Male,Writer
...,...,...,...
1537,Miles,Male,Writer
1538,Blankenship,Male,Writer
1539,Owens,Male,Writer
1540,Daniels,Male,Writer
